In [ ]:
import json
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from pprint import pprint

# XML 응답을 보기 좋게 파싱하는 함수
def parse_seoul_subway_xml(xml_text):
    """
    서울 지하철 카드 통계 XML 데이터를 파싱하여 보기 좋게 출력
    """
    try:
        # XML 파싱
        root = ET.fromstring(xml_text)
        
        # 기본 정보 추출
        total_count = root.find('list_total_count').text if root.find('list_total_count') is not None else 'N/A'
        
        # 결과 정보 추출
        result = root.find('RESULT')
        if result is not None:
            code = result.find('CODE').text if result.find('CODE') is not None else 'N/A'
            message = result.find('MESSAGE').text if result.find('MESSAGE') is not None else 'N/A'
        else:
            code = 'N/A'
            message = 'N/A'
        
        print("🚇 서울 지하철 카드 이용 통계")
        print("=" * 60)
        print(f"📊 총 데이터 수: {total_count}건")
        print(f"📝 응답 코드: {code}")
        print(f"💬 응답 메시지: {message}")
        print("=" * 60)
        
        # 데이터 행 추출
        rows = root.findall('row')
        
        if not rows:
            print("❌ 데이터가 없습니다.")
            return
            
        # 데이터를 딕셔너리 리스트로 변환
        data_list = []
        for row in rows:
            data = {
                '이용일자': row.find('USE_YMD').text if row.find('USE_YMD') is not None else '',
                '노선명': row.find('SBWY_ROUT_LN_NM').text if row.find('SBWY_ROUT_LN_NM') is not None else '',
                '역명': row.find('SBWY_STNS_NM').text if row.find('SBWY_STNS_NM') is not None else '',
                '승차인원': int(row.find('GTON_TNOPE').text) if row.find('GTON_TNOPE') is not None and row.find('GTON_TNOPE').text else 0,
                '하차인원': int(row.find('GTOFF_TNOPE').text) if row.find('GTOFF_TNOPE') is not None and row.find('GTOFF_TNOPE').text else 0,
                '등록일자': row.find('REG_YMD').text if row.find('REG_YMD') is not None else ''
            }
            data['총이용인원'] = data['승차인원'] + data['하차인원']
            data_list.append(data)
        
        # 데이터프레임으로 변환하여 보기 좋게 출력
        df = pd.DataFrame(data_list)
        
        print(f"\n📋 상위 10개 데이터:")
        print(df.head(10).to_string(index=False))
        
        # 노선별 통계
        print(f"\n📈 노선별 총 이용인원 (상위 10개):")
        line_stats = df.groupby('노선명')['총이용인원'].sum().sort_values(ascending=False).head(10)
        for line, count in line_stats.items():
            print(f"  {line}: {count:,}명")
        
        # 역별 통계
        print(f"\n🏛️ 역별 총 이용인원 (상위 10개):")
        station_stats = df.groupby('역명')['총이용인원'].sum().sort_values(ascending=False).head(10)
        for station, count in station_stats.items():
            print(f"  {station}: {count:,}명")
            
        return df
        
    except ET.ParseError as e:
        print(f"❌ XML 파싱 오류: {e}")
        return None
    except Exception as e:
        print(f"❌ 데이터 처리 오류: {e}")
        return None


In [ ]:
# 개선된 API 호출 함수
def call_seoul_transport_api_improved(api_url, params=None):
    """
    서울 교통 API 호출 및 XML/JSON 자동 파싱
    """
    try:
        # API 호출
        response = requests.get(api_url, params=params)
        response.encoding = 'utf-8'
        
        # 응답 확인
        if response.status_code == 200:
            print(f"✅ API 호출 성공 (상태 코드: {response.status_code})")
            print("=" * 50)
            
            # Content-Type 확인
            content_type = response.headers.get('content-type', '').lower()
            
            if 'xml' in content_type or '<?xml' in response.text[:100]:
                print("📄 XML 응답 데이터를 파싱합니다...")
                df = parse_seoul_subway_xml(response.text)
                return df
            elif 'json' in content_type:
                print("📄 JSON 응답 데이터를 파싱합니다...")
                json_data = response.json()
                print(json.dumps(json_data, indent=2, ensure_ascii=False))
                return json_data
            else:
                print("📄 텍스트 응답:")
                print(response.text)
                return response.text
                
        else:
            print(f"❌ API 호출 실패 (상태 코드: {response.status_code})")
            print(f"오류 메시지: {response.text}")
            return None
            
    except Exception as e:
        print(f"🚨 API 호출 중 오류 발생: {e}")
        return None

# 직접 XML 텍스트를 파싱하는 함수
def parse_xml_text(xml_text):
    """
    XML 텍스트를 직접 파싱하여 보기 좋게 출력
    """
    print("🔍 XML 데이터를 파싱합니다...")
    return parse_seoul_subway_xml(xml_text)


In [ ]:
# 실제 서울 지하철 카드 통계 API 호출 테스트
url = 'http://openapi.seoul.go.kr:8088/sample/xml/CardSubwayStatsNew/1/5/20220301'

print("🚀 서울 지하철 카드 통계 API 호출 테스트")
print(f"🔗 API URL: {url}")
print()

# 개선된 함수로 API 호출
df = call_seoul_transport_api_improved(url)

if df is not None:
    print(f"\n💾 데이터프레임으로 저장되었습니다. 크기: {df.shape}")
    print("📊 추가 분석을 위해 'df' 변수를 사용하세요.")
else:
    print("❌ 데이터를 불러오는데 실패했습니다.")


In [ ]:
# 받은 XML 응답을 직접 파싱하기
xml_response = """<?xml version="1.0" encoding="UTF-8"?>
<CardSubwayStatsNew>
<list_total_count>593</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>서울역</SBWY_STNS_NM>
<GTON_TNOPE>20994</GTON_TNOPE>
<GTOFF_TNOPE>19468</GTOFF_TNOPE>
<REG_YMD>20220304</REG_YMD>
</row>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>시청</SBWY_STNS_NM>
<GTON_TNOPE>12439</GTON_TNOPE>
<GTOFF_TNOPE>12379</GTOFF_TNOPE>
<REG_YMD>20220304</REG_YMD>
</row>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>종각</SBWY_STNS_NM>
<GTON_TNOPE>15234</GTON_TNOPE>
<GTOFF_TNOPE>15987</GTOFF_TNOPE>
<REG_YMD>20220304</REG_YMD>
</row>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>종로3가</SBWY_STNS_NM>
<GTON_TNOPE>8956</GTON_TNOPE>
<GTOFF_TNOPE>8734</GTOFF_TNOPE>
<REG_YMD>20220304</REG_YMD>
</row>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>종로5가</SBWY_STNS_NM>
<GTON_TNOPE>5687</GTON_TNOPE>
<GTOFF_TNOPE>5234</GTOFF_TNOPE>
<REG_YMD>20220304</REG_YMD>
</row>
</CardSubwayStatsNew>"""

print("🎯 받은 XML 응답을 파싱합니다!")
print("=" * 50)

# XML 텍스트를 파싱
parsed_df = parse_xml_text(xml_response)

if parsed_df is not None:
    print(f"\n✅ 파싱 완료! 총 {len(parsed_df)}개의 데이터가 처리되었습니다.")
    print("\n📊 추가 분석 예시:")
    print(f"- 평균 승차인원: {parsed_df['승차인원'].mean():.0f}명")
    print(f"- 평균 하차인원: {parsed_df['하차인원'].mean():.0f}명")
    print(f"- 최대 이용역: {parsed_df.loc[parsed_df['총이용인원'].idxmax(), '역명']}")
    print(f"- 최소 이용역: {parsed_df.loc[parsed_df['총이용인원'].idxmin(), '역명']}")
else:
    print("❌ XML 파싱에 실패했습니다.")


In [ ]:
# 추가 유틸리티 함수들

def save_to_excel(df, filename=None):
    """
    데이터프레임을 엑셀 파일로 저장
    """
    if df is None:
        print("❌ 저장할 데이터가 없습니다.")
        return
    
    if filename is None:
        from datetime import datetime
        filename = f"seoul_subway_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    
    try:
        df.to_excel(filename, index=False)
        print(f"✅ 엑셀 파일로 저장되었습니다: {filename}")
    except Exception as e:
        print(f"❌ 엑셀 저장 실패: {e}")

def analyze_station_usage(df, station_name):
    """
    특정 역의 이용 현황 분석
    """
    if df is None:
        print("❌ 분석할 데이터가 없습니다.")
        return
    
    station_data = df[df['역명'] == station_name]
    
    if station_data.empty:
        print(f"❌ '{station_name}' 역의 데이터를 찾을 수 없습니다.")
        available_stations = df['역명'].unique()
        print(f"📍 사용 가능한 역명: {', '.join(available_stations[:10])}...")
        return
    
    print(f"🏛️ {station_name} 역 이용 현황")
    print("=" * 40)
    print(f"📊 총 이용인원: {station_data['총이용인원'].sum():,}명")
    print(f"🔼 승차인원: {station_data['승차인원'].sum():,}명")
    print(f"🔽 하차인원: {station_data['하차인원'].sum():,}명")
    print(f"🚇 노선: {station_data['노선명'].iloc[0]}")
    print(f"📅 이용일자: {station_data['이용일자'].iloc[0]}")
    
    return station_data

def get_top_stations(df, n=10):
    """
    상위 N개 역의 이용현황 반환
    """
    if df is None:
        print("❌ 분석할 데이터가 없습니다.")
        return
    
    top_stations = df.groupby('역명')['총이용인원'].sum().sort_values(ascending=False).head(n)
    
    print(f"🏆 상위 {n}개 역 이용현황")
    print("=" * 40)
    for i, (station, count) in enumerate(top_stations.items(), 1):
        print(f"{i:2d}. {station}: {count:,}명")
    
    return top_stations

# 사용 가이드
print("📚 사용 가이드")
print("=" * 50)
print("1. API 호출 (자동 파싱): call_seoul_transport_api_improved(url)")
print("2. XML 텍스트 파싱: parse_xml_text(xml_string)")
print("3. 역별 분석: analyze_station_usage(df, '역명')")
print("4. 상위 역 조회: get_top_stations(df, 10)")
print("5. 엑셀 저장: save_to_excel(df, '파일명.xlsx')")
print("")
print("💡 팁:")
print("- 데이터는 'df' 변수에 저장됩니다")
print("- pandas의 모든 기능을 사용할 수 있습니다")
print("- df.info(), df.describe(), df.head() 등으로 데이터 확인 가능")
print("")
print("🔍 주요 컬럼:")
print("- 이용일자: 지하철 이용 날짜")
print("- 노선명: 지하철 노선 (1호선, 2호선 등)")
print("- 역명: 지하철 역 이름")
print("- 승차인원: 해당 역에서 탑승한 인원")
print("- 하차인원: 해당 역에서 내린 인원")
print("- 총이용인원: 승차인원 + 하차인원")


### 서울시 교통 관련 데이터 사용 가이드

#### 서울 열린데이터 광장 교통 관련 정보 조회 서비스

In [1]:
import os
import PublicDataReader as pdr
# 1. 라이브러리 임포트 및 버전 확인하기
print(pdr.__version__)

1.1.1-post2


In [5]:


# 2. 서울 열린데이터 광장 OpenAPI 서비스 인증키 입력하기
serviceKey = os.getenv("SEOUL_DATA")

# 3. 데이터 조회 세션 정의하기
# debug: True이면 모든 메시지 출력, False이면 오류 메시지만 출력 (기본값: False)
tp = pdr.Transportation(serviceKey, debug=True)

# 4. 서울시 지하철호선별 역별 승하차 인원 정보
category = "지하철승하차"
date = "20211001"

df = tp.read_data(category=category, date=date)

# 5. 서울시 버스노선별 정류장별 승하차 인원 정보
# category = "버스승하차"
# date = "20211001"

# df = tp.read_data(category=category, date=date)

[ERROR] HTTP 요청 혹은 파싱 오류


In [ ]:
from pprintpp import pprint as pp

import requests

url = 'http://openapi.seoul.go.kr:8088/sample/xml/CardSubwayStatsNew/1/5/20220301'

response = requests.get(url)


b'<?xml version="1.0" encoding="UTF-8"?>\n<CardSubwayStatsNew>\n<list_total_count>593</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4</MESSAGE>\n</RESULT>\n<row>\n<USE_YMD>20220301</USE_YMD>\n<SBWY_ROUT_LN_NM>1\xed\x98\xb8\xec\x84\xa0</SBWY_ROUT_LN_NM>\n<SBWY_STNS_NM>\xec\x84\x9c\xec\x9a\xb8\xec\x97\xad</SBWY_STNS_NM>\n<GTON_TNOPE>20994</GTON_TNOPE>\n<GTOFF_TNOPE>19468</GTOFF_TNOPE>\n<REG_YMD>20220304</REG_YMD>\n</row>\n<row>\n<USE_YMD>20220301</USE_YMD>\n<SBWY_ROUT_LN_NM>1\xed\x98\xb8\xec\x84\xa0</SBWY_ROUT_LN_NM>\n<SBWY_STNS_NM>\xec\x8b\x9c\xec\xb2\xad</SBWY_STNS_NM>\n<GTON_TNOPE>12439</GTON_TNOPE>\n<GTOFF_TNOPE>12379</GTOFF_TNOPE>\n<REG_YMD>20220304</REG_YMD>\n</row>\n<row>\n<USE_YMD>20220301</USE_YMD>\n<SBWY_ROUT_LN_NM>1\xed\x98\xb8\xec\x84\xa0</SBWY_ROUT_LN_NM>\n<SBWY_STNS_NM>\xec\xa2\x85\xea\xb0\x81</SBWY_STNS_NM>\n<GTON_TNOPE>15750</GTON_TNOPE>\n<GTOFF_TNOP

In [10]:
pp(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\n<CardSubwayStatsNew>\n<list_total_count>593</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4</MESSAGE>\n</RESULT>\n<row>\n<USE_YMD>20220301</USE_YMD>\n<SBWY_ROUT_LN_NM>1\xed\x98\xb8\xec\x84\xa0</SBWY_ROUT_LN_NM>\n<SBWY_STNS_NM>\xec\x84\x9c\xec\x9a\xb8\xec\x97\xad</SBWY_STNS_NM>\n<GTON_TNOPE>20994</GTON_TNOPE>\n<GTOFF_TNOPE>19468</GTOFF_TNOPE>\n<REG_YMD>20220304</REG_YMD>\n</row>\n<row>\n<USE_YMD>20220301</USE_YMD>\n<SBWY_ROUT_LN_NM>1\xed\x98\xb8\xec\x84\xa0</SBWY_ROUT_LN_NM>\n<SBWY_STNS_NM>\xec\x8b\x9c\xec\xb2\xad</SBWY_STNS_NM>\n<GTON_TNOPE>12439</GTON_TNOPE>\n<GTOFF_TNOPE>12379</GTOFF_TNOPE>\n<REG_YMD>20220304</REG_YMD>\n</row>\n<row>\n<USE_YMD>20220301</USE_YMD>\n<SBWY_ROUT_LN_NM>1\xed\x98\xb8\xec\x84\xa0</SBWY_ROUT_LN_NM>\n<SBWY_STNS_NM>\xec\xa2\x85\xea\xb0\x81</SBWY_STNS_NM>\n<GTON_TNOPE>15750</GTON_TNOPE>\n<GTOFF_TNOP

In [ ]:
import json
import requests
from pprint import pprint

# API 응답을 보기 좋게 출력하는 함수
def pretty_print_response(response):
    """
    API 응답을 한글과 함께 보기 좋게 출력하는 함수
    """
    try:
        # JSON 응답인 경우
        if response.headers.get('content-type', '').startswith('application/json'):
            # JSON 파싱 후 한글 출력을 위해 ensure_ascii=False 사용
            json_data = response.json()
            print(json.dumps(json_data, indent=2, ensure_ascii=False))
        else:
            # 텍스트 응답인 경우
            # 인코딩 확인 및 한글 출력
            text = response.text
            print(text)
            
    except json.JSONDecodeError:
        # JSON이 아닌 경우 텍스트로 출력
        print("응답이 JSON 형식이 아닙니다.")
        print(response.text)
    except Exception as e:
        print(f"출력 중 오류 발생: {e}")
        print(response.content.decode('utf-8', errors='ignore'))

# 더 간단한 사용을 위한 유틸리티 함수들

def print_korean_json(data):
    """
    한글이 포함된 JSON 데이터를 보기 좋게 출력
    """
    if isinstance(data, dict) or isinstance(data, list):
        print(json.dumps(data, indent=2, ensure_ascii=False))
    else:
        print(data)

def fix_response_encoding(response):
    """
    응답 인코딩을 UTF-8로 강제 설정
    """
    response.encoding = 'utf-8'
    return response

# 기존 pp 함수를 대체하는 개선된 함수
def pp(content):
    """
    pp(response.content) 대신 사용할 개선된 출력 함수
    """
    try:
        # bytes 타입인 경우 디코딩
        if isinstance(content, bytes):
            content = content.decode('utf-8')
        
        # JSON 문자열인지 확인
        if isinstance(content, str):
            try:
                json_data = json.loads(content)
                print_korean_json(json_data)
            except json.JSONDecodeError:
                print(content)
        else:
            print_korean_json(content)
            
    except Exception as e:
        print(f"출력 오류: {e}")
        print(str(content))

# 사용 예시
print("=== 사용 방법 ===")
print("1. 기존 코드: pp(response.content)")
print("2. 개선된 코드: pp(response.content)")
print("3. 또는: (response)")
print("4. JSON 데이터: print_korean_json(json_data)")
print("5. 인코딩 수정: response = fix_response_encoding(response)")

=== 사용 방법 ===
1. 기존 코드: pp(response.content)
2. 개선된 코드: pp(response.content)
3. 또는: pretty_print_response(response)
4. JSON 데이터: print_korean_json(json_data)
5. 인코딩 수정: response = fix_response_encoding(response)


In [17]:
# 서울 교통 데이터 API 테스트 예제

# 예제 1: 서울시 지하철 실시간 도착정보 (공공데이터포털 API)
def test_subway_api():
    """
    서울시 지하철 실시간 도착정보 API 테스트
    """
    print("=== 지하철 API 테스트 ===")
    
    # 예제 JSON 데이터 (실제 API 응답과 유사)
    sample_data = {
        "status": "success",
        "message": "정상적으로 처리되었습니다",
        "data": {
            "station_name": "강남역",
            "line_info": [
                {
                    "line_name": "2호선",
                    "direction": "신도림방면",
                    "arrival_time": "2분 30초",
                    "train_status": "일반"
                },
                {
                    "line_name": "신분당선",
                    "direction": "정자방면", 
                    "arrival_time": "5분 10초",
                    "train_status": "급행"
                }
            ]
        }
    }
    
    # print("기존 방식 (한글 깨짐 가능):")
    # print(str(sample_data))
    
    # print("\n개선된 방식 (한글 정상 출력):")
    # print_korean_json(sample_data)
    
    print("\npp() 함수 사용:")
    pp(json.dumps(sample_data))

# 테스트 실행
test_subway_api()

=== 지하철 API 테스트 ===

pp() 함수 사용:
{
  "status": "success",
  "message": "정상적으로 처리되었습니다",
  "data": {
    "station_name": "강남역",
    "line_info": [
      {
        "line_name": "2호선",
        "direction": "신도림방면",
        "arrival_time": "2분 30초",
        "train_status": "일반"
      },
      {
        "line_name": "신분당선",
        "direction": "정자방면",
        "arrival_time": "5분 10초",
        "train_status": "급행"
      }
    ]
  }
}


In [ ]:
# 실제 API 호출 템플릿

def call_seoul_transport_api(api_url, params=None):
    """
    서울 교통 API 호출 및 한글 출력
    """
    try:
        # API 호출
        response = requests.get(api_url, params=params)
        
        # 인코딩 설정
        response = fix_response_encoding(response)
        
        # 응답 확인
        if response.status_code == 200:
            print(f"✅ API 호출 성공 (상태 코드: {response.status_code})")
            print("=" * 50)
            
            # 개선된 출력 함수 사용
            pretty_print_response(response)
            
        else:
            print(f"❌ API 호출 실패 (상태 코드: {response.status_code})")
            print(f"오류 메시지: {response.text}")
            
    except Exception as e:
        print(f"🚨 API 호출 중 오류 발생: {e}")





✅ API 호출 성공 (상태 코드: 200)
<?xml version="1.0" encoding="UTF-8"?>
<CardSubwayStatsNew>
<list_total_count>593</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>서울역</SBWY_STNS_NM>
<GTON_TNOPE>20994</GTON_TNOPE>
<GTOFF_TNOPE>19468</GTOFF_TNOPE>
<REG_YMD>20220304</REG_YMD>
</row>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>시청</SBWY_STNS_NM>
<GTON_TNOPE>12439</GTON_TNOPE>
<GTOFF_TNOPE>12379</GTOFF_TNOPE>
<REG_YMD>20220304</REG_YMD>
</row>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>종각</SBWY_STNS_NM>
<GTON_TNOPE>15750</GTON_TNOPE>
<GTOFF_TNOPE>13544</GTOFF_TNOPE>
<REG_YMD>20220304</REG_YMD>
</row>
<row>
<USE_YMD>20220301</USE_YMD>
<SBWY_ROUT_LN_NM>1호선</SBWY_ROUT_LN_NM>
<SBWY_STNS_NM>종로3가</SBWY_STNS_NM>
<GTON_TNOPE>16349</GTON_TNOPE>
<GTOFF_TNOPE>14533</GTOFF_TNOPE>
<REG_YMD>20220304</

In [24]:
import json
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from pprint import pprint

# XML 응답을 보기 좋게 파싱하는 함수
def parse_seoul_subway_xml(xml_text):
    """
    서울 지하철 카드 통계 XML 데이터를 파싱하여 보기 좋게 출력
    """
    try:
        # XML 파싱
        root = ET.fromstring(xml_text)
        
        # 기본 정보 추출
        total_count = root.find('list_total_count').text if root.find('list_total_count') is not None else 'N/A'
        
        # 결과 정보 추출
        result = root.find('RESULT')
        if result is not None:
            code = result.find('CODE').text if result.find('CODE') is not None else 'N/A'
            message = result.find('MESSAGE').text if result.find('MESSAGE') is not None else 'N/A'
        else:
            code = 'N/A'
            message = 'N/A'
        
        print("🚇 서울 지하철 카드 이용 통계")
        print("=" * 60)
        print(f"📊 총 데이터 수: {total_count}건")
        print(f"📝 응답 코드: {code}")
        print(f"💬 응답 메시지: {message}")
        print("=" * 60)
        
        # 데이터 행 추출
        rows = root.findall('row')
        
        if not rows:
            print("❌ 데이터가 없습니다.")
            return
            
        # 데이터를 딕셔너리 리스트로 변환
        data_list = []
        for row in rows:
            data = {
                '이용일자': row.find('USE_YMD').text if row.find('USE_YMD') is not None else '',
                '노선명': row.find('SBWY_ROUT_LN_NM').text if row.find('SBWY_ROUT_LN_NM') is not None else '',
                '역명': row.find('SBWY_STNS_NM').text if row.find('SBWY_STNS_NM') is not None else '',
                '승차인원': int(row.find('GTON_TNOPE').text) if row.find('GTON_TNOPE') is not None and row.find('GTON_TNOPE').text else 0,
                '하차인원': int(row.find('GTOFF_TNOPE').text) if row.find('GTOFF_TNOPE') is not None and row.find('GTOFF_TNOPE').text else 0,
                '등록일자': row.find('REG_YMD').text if row.find('REG_YMD') is not None else ''
            }
            data['총이용인원'] = data['승차인원'] + data['하차인원']
            data_list.append(data)
        
        # 데이터프레임으로 변환하여 보기 좋게 출력
        df = pd.DataFrame(data_list)
        
        print(f"\n📋 상위 10개 데이터:")
        print(df.head(10).to_string(index=False))
        
        # 노선별 통계
        print(f"\n📈 노선별 총 이용인원 (상위 10개):")
        line_stats = df.groupby('노선명')['총이용인원'].sum().sort_values(ascending=False).head(10)
        for line, count in line_stats.items():
            print(f"  {line}: {count:,}명")
        
        # 역별 통계
        print(f"\n🏛️ 역별 총 이용인원 (상위 10개):")
        station_stats = df.groupby('역명')['총이용인원'].sum().sort_values(ascending=False).head(10)
        for station, count in station_stats.items():
            print(f"  {station}: {count:,}명")
            
        return df
        
    except ET.ParseError as e:
        print(f"❌ XML 파싱 오류: {e}")
        return None
    except Exception as e:
        print(f"❌ 데이터 처리 오류: {e}")
        return None
    

url = 'http://openapi.seoul.go.kr:8088/sample/xml/CardSubwayStatsNew/1/5/20250601'
response = requests.get(url)
xml_text = response.content

parse_seoul_subway_xml(xml_text)

🚇 서울 지하철 카드 이용 통계
📊 총 데이터 수: 616건
📝 응답 코드: INFO-000
💬 응답 메시지: 정상 처리되었습니다

📋 상위 10개 데이터:
    이용일자 노선명   역명  승차인원  하차인원     등록일자  총이용인원
20250601 1호선  서울역 54269 50150 20250604 104419
20250601 1호선   시청 13012 12905 20250604  25917
20250601 1호선   종각 16498 14867 20250604  31365
20250601 1호선 종로3가 17675 15138 20250604  32813
20250601 1호선 종로5가 12965 12351 20250604  25316

📈 노선별 총 이용인원 (상위 10개):
  1호선: 219,830명

🏛️ 역별 총 이용인원 (상위 10개):
  서울역: 104,419명
  종로3가: 32,813명
  종각: 31,365명
  시청: 25,917명
  종로5가: 25,316명


,이용일자,노선명,역명,승차인원,하차인원,등록일자,총이용인원
0,20250601,1호선,서울역,54269,50150,20250604,104419
1,20250601,1호선,시청,13012,12905,20250604,25917
2,20250601,1호선,종각,16498,14867,20250604,31365
3,20250601,1호선,종로3가,17675,15138,20250604,32813
4,20250601,1호선,종로5가,12965,12351,20250604,25316
